In [1]:
import numpy as np
import pandas as pd

In [2]:
from classification_model.processing.data_management import load_dataset
train = load_dataset(file_name='AWID-CLS-R-Trn.csv')

C:\Users\skyst\Anaconda3\envs\tf\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)
C:\Users\skyst\Anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (82,86,87,103,104,105,108,109,111,113,114,124,139,141,142,143,147) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [3]:
from classification_model.processing.data_management import downsample
train_ds = downsample(train)

In [4]:
from classification_model.processing.data_management import get_target
X_train, y_train = get_target(train_ds)

In [5]:
test = load_dataset(file_name='AWID-CLS-R-Tst.csv')
X_test, y_test = get_target(test) 

C:\Users\skyst\Anaconda3\envs\tf\lib\site-packages\IPython\core\interactiveshell.py:3254: DtypeWarning: Columns (103,104,105,108,109,111,113,114,124,142,143) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [6]:
from sklearn.model_selection import PredefinedSplit
x = pd.concat([X_train, X_test])
y = pd.concat([y_train, y_test])

test_fold = np.concatenate([
    # The training data.
    np.full( X_train.shape[0], -1),
    # The development data.
    np.zeros(X_test.shape[0])
])

cv = PredefinedSplit(test_fold)

In [7]:
param_grid = {
    #'rle__tol': [0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001]
    #'ani__arbitrary_number': [-999,-1],
    #'le__encoding_method': ['arbitrary', 'both'],
    #'rqc__threshold': [0.01,0.001],
    #'rcf__threshold': [ 0.8, 0.9, 0.95, 0.99, 0.999, 0.9999],
    'oc__distribution': ['gaussian', 'skewed', 'quantiles']
}

In [8]:
from classification_model.pipeline import xgboost_pipe
from sklearn.model_selection import GridSearchCV

models = GridSearchCV(xgboost_pipe, param_grid, cv=cv, n_jobs=1, refit = False, verbose=1)

In [9]:
models.fit(x,y)

Fitting 1 folds for each of 3 candidates, totalling 3 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed: 197.8min finished


GridSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  0,  0])),
             error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('fc',
                                        feature_creation(features=[None])),
                                       ('rle',
                                        rare_label_encoder(features=[None],
                                                           tol=0.0001)),
                                       ('cme',
                                        categ_missing_encoder(features=[None])),
                                       ('ani',
                                        ArbitraryNumberImputer(arbitrary_number=-999,
                                                               features=[None])),
                                       ('le',
                                        label_encod...
                                                      objective='multi:softmax',
         

In [10]:
pd.set_option('display.max_rows', 500)
models.best_params_

{'oc__distribution': 'quantiles'}

In [12]:
models.best_score_

0.9573050611317451